<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_7_Initialize_Traders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 引入依赖包

In [1]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib inline

# 资源类

In [2]:
class Sugar(mesa.Agent):
    '''
    Sugar:
    - 包含一定数量的糖
    - 每轮增长一单位数量的糖
    '''

    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model)
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

In [3]:
class Spice(mesa.Agent):
  '''
  spice:
  - 包含一定数量的香料
  - 每轮增长一单位数量的香料
  '''

  def __init__(self, unique_id, model, pos, max_spice):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_spice
    self.max_spice = max_spice
  

# 交易主体类

In [4]:
class Trader(mesa.Agent): 
    '''
    Trader:
    - 能够代谢糖和香料
    - 收集和交易糖和香料来生存与繁衍
    '''


    def __init__(self, unique_id, model, pos, moore=False, sugar=0, 
                spice=0, metabolism_sugar=0, metabolism_spice=0, 
                vision=0):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore  # 是否使用Moore邻居
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision
    


# Model Class

In [5]:
class SugarscapeG1mt(mesa.Model):
  '''
  通过Traders (GImt)来运行Sugarscape的模型
  来自 Axtell和Epstein《Growing Artifical Societies》
  '''
  
  
  def __init__(self, width=50,height=50, initial_population=200,
               endowment_min=25, endowment_max=50, metabolism_min=1,
               metabolism_max=5, vision_min=1, vision_max=5):
    
    # 初始化糖景的宽度和高度
    self.width = width
    self.height = height 
    # 初始化交易主体属性
    self.initial_population = initial_population
    self.endowment_min = endowment_min
    self.endowment_max = endowment_max
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max

    # 初始化mesa的日程表
    self.schedule = mesa.time.RandomActivationByType(self)    
    # 初始化mesa的网格类
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    # 从辅助资料中读取糖和香料的分布
    sugar_distribution =np.genfromtxt("sugar-map.txt")
    spice_distribution = np.flip(sugar_distribution, 1)
    
    agent_id = 0
    for _,x,y in self.grid.coord_iter():
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y), max_sugar)
        self.schedule.add(sugar)
        self.grid.place_agent(sugar, (x,y))
        agent_id += 1
    
      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.schedule.add(spice)
        self.grid.place_agent(spice, (x,y))
        agent_id += 1    

    for i in range(self.initial_population):
      # 得到随机的坐标
      x = self.random.randrange(self.width)
      y = self.random.randrange(self.height)
      # see Growing Artificial Societies p. 108 for initialization
      # 给予主体初始的禀赋
      sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      # 给予主体初始代谢
      metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      # 给予主体视野
      vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
      # 实例化交易主体
      trader = Trader(agent_id, 
                      self,
                      (x,y),
                      moore = False, 
                      sugar = sugar, 
                      spice = spice, 
                      metabolism_sugar = metabolism_sugar, 
                      metabolism_spice = metabolism_spice, 
                      vision = vision)
      # 放置主体到网格中
      self.grid.place_agent(trader, (x,y))
      self.schedule.add(trader)
      print(trader.unique_id, trader.pos, trader.sugar, trader.metabolism_spice)
      agent_id += 1


# 运行Sugarscape模型

In [6]:
model = SugarscapeG1mt()



4138 (27, 16) 39 5
4139 (45, 45) 50 1
4140 (21, 17) 39 3
4141 (9, 31) 33 1
4142 (37, 37) 50 4
4143 (7, 23) 46 1
4144 (13, 34) 30 2
4145 (36, 37) 44 5
4146 (37, 4) 47 1
4147 (40, 47) 38 3
4148 (1, 49) 30 4
4149 (7, 31) 28 1
4150 (46, 16) 27 4
4151 (16, 20) 29 3
4152 (29, 18) 44 5
4153 (47, 2) 42 5
4154 (29, 10) 50 3
4155 (34, 17) 49 2
4156 (49, 47) 49 2
4157 (47, 2) 28 4
4158 (10, 0) 40 4
4159 (6, 19) 49 2
4160 (48, 30) 35 3
4161 (47, 6) 42 3
4162 (14, 24) 39 3
4163 (40, 24) 36 1
4164 (8, 11) 25 5
4165 (25, 25) 40 4
4166 (39, 20) 47 1
4167 (6, 5) 39 1
4168 (17, 21) 25 4
4169 (29, 4) 31 2
4170 (17, 14) 35 5
4171 (46, 23) 44 2
4172 (44, 2) 26 3
4173 (34, 11) 26 3
4174 (4, 21) 49 2
4175 (24, 33) 40 5
4176 (16, 26) 46 3
4177 (21, 42) 25 4
4178 (19, 24) 46 4
4179 (44, 16) 26 3
4180 (27, 33) 43 4
4181 (32, 13) 43 3
4182 (2, 26) 34 2
4183 (48, 30) 40 3
4184 (49, 30) 31 4
4185 (18, 31) 37 2
4186 (27, 9) 30 4
4187 (21, 27) 32 4
4188 (41, 41) 25 1
4189 (28, 47) 30 5
4190 (28, 4) 49 3
4191 (22, 0)